## Big Data Ingestion Pipeline

We are performing below operations


* Reading RAW data from REST API
* Add metadata information to RAW data
* Load RAW data into landing storage
* (optionally) create table to query this landing data.
* Reading landing data from landing storage
* Flatning the data to be alligned with staging requirements
* Load data into staging storage 
* (optionally) create table to query this staging data.
* Reading staging data from staging storage
* Apply UPSERT (Update Or Insert) w.r.t processed data.
* (optionally) create table to query this processed data.

#### Import Configuration

In [0]:
%run ../includes/configuration

Out[2]: DataFrame[]

#### Import Utility Functions

In [0]:
%run ../includes/util

Out[28]: DataFrame[]

#### Make Notebook Idempotent

In [0]:
# removes file from ingestion parent path to be able to independently run notebook for any user.
dbutils.fs.rm(ingestionPipelinePath, recurse=True)

Out[12]: True

In [0]:
# Read raw data from REST API
raw_df = read_api_raw(spark, restApiEndpoint)

Wrote 14872 bytes.


In [0]:
display(raw_df)

raw_data


In [0]:
# add metadata to raw dataframe
raw_df = add_metadata_raw(raw_df)

In [0]:
dataframe_writer = create_dataframe_writer(raw_df, 'p_ingestdate')
dataframe_writer.save(landingPath)
# remove temp file, to make notebook idempotent
dbutils.fs.rm(ingestionPipelinePath+'temp/api_raw_data.json', recurse=True)

Out[18]: True

In [0]:
# add landing data to metastore, so that it can be queries
add_to_metastore(table_name='bike_details_landing', path=landingPath)

#### Verify if table registered in metastore

In [0]:
%sql

SELECT * FROM bike_details_landing

datasource ingesttime raw_data p_ingestdate https://bikeindex.org/api/v3/search 2022-02-08T11:43:48.917+0000 List(List(1644300653, null, null, List(Orange, Black), Fuel Ex 9.8xt, 1216884, false, null, null, Trek, null, null, WTU154CT0044S, stolen, true, List(47.59, -122.04), Sammamish, WA 98075, US, null, 2022 Trek Fuel Ex 9.8xt, https://bikeindex.org/bikes/1216884, 2022), List(1644289424, null, null, List(White), jynx, 1216864, false, https://files.bikeindex.org/uploads/Pu/513452/large_1C4E6606-A035-4BEF-B5D5-1B0C14257FFC.jpeg, null, Specialized, null, null, wsbc606078869j, stolen, true, List(37.76, -122.48), San Francisco, CA 94122, US, https://files.bikeindex.org/uploads/Pu/513452/small_1C4E6606-A035-4BEF-B5D5-1B0C14257FFC.jpeg, Specialized jynx, https://bikeindex.org/bikes/1216864, null), List(1644289418, null, null, List(Blue), Hotrock, 1216865, false, null, null, Specialized, null, null, Unknown, stolen, true, List(39.73, -104.92), Denver, CO 80220, US, null, 2019 Specialized Hotrock, https://bikeindex.org/bikes/1216865, 2019), List(1644294068, Well used, well loved MTB. 

Has black grips instead of the blue grips in the photo.

Has a "Nomad Cyclclery" shop sticker - our local bike shop in San Francisco , null, List(White), Lynx, 1216869, false, https://files.bikeindex.org/uploads/Pu/513463/large_Specialized_Jynx_2015.jpeg, null, Specialized, null, null, WSBC606078869J, stolen, true, null, US, https://files.bikeindex.org/uploads/Pu/513463/small_Specialized_Jynx_2015.jpeg, 2015 Specialized Lynx, https://bikeindex.org/bikes/1216869, 2015), List(null, null, null, List(Black), Escape 3, 1216787, false, null, 5711 S 129th St, Seattle, WA 98178, US, Giant, null, null, Hidden, found, false, null, null, null, Giant Escape 3, https://bikeindex.org/bikes/1216787, null), List(1644273561, null, null, List(Green), Kuma 27.2, 1216800, false, null, null, Bianchi, null, null, wbk832800l, stolen, true, List(47.63, -122.35), Seattle, WA 98109, US, null, 2019 Bianchi Kuma 27.2, https://bikeindex.org/bikes/1216800, 2019), List(1644271200, null, null, List(Blue), Quick 5, 1216792, false, https://files.bikeindex.org/uploads/Pu/513385/large_Cannondale_Quick_5_-_Electric_Blue.png, null, Cannondale, null, null, YC63733, stolen, true, List(47.67, -122.19), Kirkland, WA 98033, US, https://files.bikeindex.org/uploads/Pu/513385/small_Cannondale_Quick_5_-_Electric_Blue.png, Cannondale Quick 5, https://bikeindex.org/bikes/1216792, null), List(1644271200, null, null, List(Black), Quick Disc LTD, 1216798, false, https://files.bikeindex.org/uploads/Pu/513386/large_Cannondale_Quick_Disc_LTD_-_Black.jpeg, null, Cannondale, null, null, YC06366, stolen, true, List(47.67, -122.19), Kirkland, WA 98033, US, https://files.bikeindex.org/uploads/Pu/513386/small_Cannondale_Quick_Disc_LTD_-_Black.jpeg, Cannondale Quick Disc LTD, https://bikeindex.org/bikes/1216798, null), List(1644269041, null, null, List(Brown, Black), S1, 1216765, false, https://files.bikeindex.org/uploads/Pu/513361/large_Screen_Shot_2022-02-07_at_1.27.25_PM.png, null, Super 73, null, null, KR1041Y0936, stolen, true, List(33.86, -118.4), Hermosa Beach, CA 90254, US, https://files.bikeindex.org/uploads/Pu/513361/small_Screen_Shot_2022-02-07_at_1.27.25_PM.png, 2019 Super 73 S1, https://bikeindex.org/bikes/1216765, 2019), List(1644266739, null, null, List(Blue), Model H, 1216777, false, null, null, HeartsBio, null, null, Unknown, stolen, true, List(37.78, -122.5), San Francisco, CA 94121, US, null, 2020 HeartsBio Model H, https://bikeindex.org/bikes/1216777, 2020), List(1644267105, midnight special 50 sour strawberry sparkle, null, List(Red), midnight special 50 sour strawberry sparkle, 1104604, false, https://files.bikeindex.org/uploads/Pu/513358/large_F5626020-9553-4814-B99F-430D48A2FE63.jpeg, null, Surly, null, null, M21024010, stolen, true, null, US, https://files.bikeindex.org/uploads/Pu/513358/small_F5626020-9553-4814-B99F-430D48A2FE63.jpeg, 2021 Surly midnight special 50 sour strawberry s.

In [0]:
display(dbutils.fs.ls(landingPath))

path,name,size
dbfs:/ingestion/dataengineering/gsg/raw/_delta_log/,_delta_log/,0
dbfs:/ingestion/dataengineering/gsg/raw/p_ingestdate=2022-02-08/,p_ingestdate=2022-02-08/,0


In [0]:
dbutils.fs.rm(stagingPath, recurse=True)

Out[21]: False

In [0]:
from datetime import datetime
# read landing data
landing_df = spark.read.table("bike_details_landing").filter("p_ingestdate == '{}'".format(datetime.today().strftime('%Y-%m-%d')))

In [0]:
display(landing_df)

datasource ingesttime raw_data p_ingestdate https://bikeindex.org/api/v3/search 2022-02-08T11:43:48.917+0000 List(List(1644300653, null, null, List(Orange, Black), Fuel Ex 9.8xt, 1216884, false, null, null, Trek, null, null, WTU154CT0044S, stolen, true, List(47.59, -122.04), Sammamish, WA 98075, US, null, 2022 Trek Fuel Ex 9.8xt, https://bikeindex.org/bikes/1216884, 2022), List(1644289424, null, null, List(White), jynx, 1216864, false, https://files.bikeindex.org/uploads/Pu/513452/large_1C4E6606-A035-4BEF-B5D5-1B0C14257FFC.jpeg, null, Specialized, null, null, wsbc606078869j, stolen, true, List(37.76, -122.48), San Francisco, CA 94122, US, https://files.bikeindex.org/uploads/Pu/513452/small_1C4E6606-A035-4BEF-B5D5-1B0C14257FFC.jpeg, Specialized jynx, https://bikeindex.org/bikes/1216864, null), List(1644289418, null, null, List(Blue), Hotrock, 1216865, false, null, null, Specialized, null, null, Unknown, stolen, true, List(39.73, -104.92), Denver, CO 80220, US, null, 2019 Specialized Hotrock, https://bikeindex.org/bikes/1216865, 2019), List(1644294068, Well used, well loved MTB. 

Has black grips instead of the blue grips in the photo.

Has a "Nomad Cyclclery" shop sticker - our local bike shop in San Francisco , null, List(White), Lynx, 1216869, false, https://files.bikeindex.org/uploads/Pu/513463/large_Specialized_Jynx_2015.jpeg, null, Specialized, null, null, WSBC606078869J, stolen, true, null, US, https://files.bikeindex.org/uploads/Pu/513463/small_Specialized_Jynx_2015.jpeg, 2015 Specialized Lynx, https://bikeindex.org/bikes/1216869, 2015), List(null, null, null, List(Black), Escape 3, 1216787, false, null, 5711 S 129th St, Seattle, WA 98178, US, Giant, null, null, Hidden, found, false, null, null, null, Giant Escape 3, https://bikeindex.org/bikes/1216787, null), List(1644273561, null, null, List(Green), Kuma 27.2, 1216800, false, null, null, Bianchi, null, null, wbk832800l, stolen, true, List(47.63, -122.35), Seattle, WA 98109, US, null, 2019 Bianchi Kuma 27.2, https://bikeindex.org/bikes/1216800, 2019), List(1644271200, null, null, List(Blue), Quick 5, 1216792, false, https://files.bikeindex.org/uploads/Pu/513385/large_Cannondale_Quick_5_-_Electric_Blue.png, null, Cannondale, null, null, YC63733, stolen, true, List(47.67, -122.19), Kirkland, WA 98033, US, https://files.bikeindex.org/uploads/Pu/513385/small_Cannondale_Quick_5_-_Electric_Blue.png, Cannondale Quick 5, https://bikeindex.org/bikes/1216792, null), List(1644271200, null, null, List(Black), Quick Disc LTD, 1216798, false, https://files.bikeindex.org/uploads/Pu/513386/large_Cannondale_Quick_Disc_LTD_-_Black.jpeg, null, Cannondale, null, null, YC06366, stolen, true, List(47.67, -122.19), Kirkland, WA 98033, US, https://files.bikeindex.org/uploads/Pu/513386/small_Cannondale_Quick_Disc_LTD_-_Black.jpeg, Cannondale Quick Disc LTD, https://bikeindex.org/bikes/1216798, null), List(1644269041, null, null, List(Brown, Black), S1, 1216765, false, https://files.bikeindex.org/uploads/Pu/513361/large_Screen_Shot_2022-02-07_at_1.27.25_PM.png, null, Super 73, null, null, KR1041Y0936, stolen, true, List(33.86, -118.4), Hermosa Beach, CA 90254, US, https://files.bikeindex.org/uploads/Pu/513361/small_Screen_Shot_2022-02-07_at_1.27.25_PM.png, 2019 Super 73 S1, https://bikeindex.org/bikes/1216765, 2019), List(1644266739, null, null, List(Blue), Model H, 1216777, false, null, null, HeartsBio, null, null, Unknown, stolen, true, List(37.78, -122.5), San Francisco, CA 94121, US, null, 2020 HeartsBio Model H, https://bikeindex.org/bikes/1216777, 2020), List(1644267105, midnight special 50 sour strawberry sparkle, null, List(Red), midnight special 50 sour strawberry sparkle, 1104604, false, https://files.bikeindex.org/uploads/Pu/513358/large_F5626020-9553-4814-B99F-430D48A2FE63.jpeg, null, Surly, null, null, M21024010, stolen, true, null, US, https://files.bikeindex.org/uploads/Pu/513358/small_F5626020-9553-4814-B99F-430D48A2FE63.jpeg, 2021 Surly midnight special 50 sour strawberry s.

In [0]:
# Transform landing data
transformed_df = transform_landing(landing_df)

In [0]:
display(transformed_df)

date_stolen,description,external_id,frame_colors,frame_model,id,is_stock_img,large_img,location_found,manufacturer_name,registry_name,registry_url,serial,status,stolen,stolen_coordinates,stolen_location,thumb,title,url,year,p_incident_date
1644300653,null,null,"List(Orange, Black)",Fuel Ex 9.8xt,1216884,false,null,null,Trek,null,null,WTU154CT0044S,stolen,true,"List(47.59, -122.04)","Sammamish, WA 98075, US",null,2022 Trek Fuel Ex 9.8xt,https://bikeindex.org/bikes/1216884,2022,2022-02-08
1644289424,null,null,List(White),jynx,1216864,false,https://files.bikeindex.org/uploads/Pu/513452/large_1C4E6606-A035-4BEF-B5D5-1B0C14257FFC.jpeg,null,Specialized,null,null,wsbc606078869j,stolen,true,"List(37.76, -122.48)","San Francisco, CA 94122, US",https://files.bikeindex.org/uploads/Pu/513452/small_1C4E6606-A035-4BEF-B5D5-1B0C14257FFC.jpeg,Specialized jynx,https://bikeindex.org/bikes/1216864,null,2022-02-08
1644289418,null,null,List(Blue),Hotrock,1216865,false,null,null,Specialized,null,null,Unknown,stolen,true,"List(39.73, -104.92)","Denver, CO 80220, US",null,2019 Specialized Hotrock,https://bikeindex.org/bikes/1216865,2019,2022-02-08
1644294068,"Well used, well loved MTB. Has black grips instead of the blue grips in the photo. Has a ""Nomad Cyclclery"" shop sticker - our local bike shop in San Francisco",null,List(White),Lynx,1216869,false,https://files.bikeindex.org/uploads/Pu/513463/large_Specialized_Jynx_2015.jpeg,null,Specialized,null,null,WSBC606078869J,stolen,true,null,US,https://files.bikeindex.org/uploads/Pu/513463/small_Specialized_Jynx_2015.jpeg,2015 Specialized Lynx,https://bikeindex.org/bikes/1216869,2015,2022-02-08
null,null,null,List(Black),Escape 3,1216787,false,null,"5711 S 129th St, Seattle, WA 98178, US",Giant,null,null,Hidden,found,false,null,null,null,Giant Escape 3,https://bikeindex.org/bikes/1216787,null,2022-02-08
1644273561,null,null,List(Green),Kuma 27.2,1216800,false,null,null,Bianchi,null,null,wbk832800l,stolen,true,"List(47.63, -122.35)","Seattle, WA 98109, US",null,2019 Bianchi Kuma 27.2,https://bikeindex.org/bikes/1216800,2019,2022-02-08
1644271200,null,null,List(Blue),Quick 5,1216792,false,https://files.bikeindex.org/uploads/Pu/513385/large_Cannondale_Quick_5_-_Electric_Blue.png,null,Cannondale,null,null,YC63733,stolen,true,"List(47.67, -122.19)","Kirkland, WA 98033, US",https://files.bikeindex.org/uploads/Pu/513385/small_Cannondale_Quick_5_-_Electric_Blue.png,Cannondale Quick 5,https://bikeindex.org/bikes/1216792,null,2022-02-08
1644271200,null,null,List(Black),Quick Disc LTD,1216798,false,https://files.bikeindex.org/uploads/Pu/513386/large_Cannondale_Quick_Disc_LTD_-_Black.jpeg,null,Cannondale,null,null,YC06366,stolen,true,"List(47.67, -122.19)","Kirkland, WA 98033, US",https://files.bikeindex.org/uploads/Pu/513386/small_Cannondale_Quick_Disc_LTD_-_Black.jpeg,Cannondale Quick Disc LTD,https://bikeindex.org/bikes/1216798,null,2022-02-08
1644269041,null,null,"List(Brown, Black)",S1,1216765,false,https://files.bikeindex.org/uploads/Pu/513361/large_Screen_Shot_2022-02-07_at_1.27.25_PM.png,null,Super 73,null,null,KR1041Y0936,stolen,true,"List(33.86, -118.4)","Hermosa Beach, CA 90254, US",https://files.bikeindex.org/uploads/Pu/513361/small_Screen_Shot_2022-02-07_at_1.27.25_PM.png,2019 Super 73 S1,https://bikeindex.org/bikes/1216765,2019,2022-02-08
1644266739,null,null,List(Blue),Model H,1216777,false,null,null,HeartsBio,null,null,Unknown,stolen,true,"List(37.78, -122.5)","San Francisco, CA 94121, US",null,2020 HeartsBio Model H,https://bikeindex.org/bikes/1216777,2020,2022-02-08


In [0]:
dataframe_writer = create_dataframe_writer(transformed_df, 'p_incident_date')
dataframe_writer.save(stagingPath)

In [0]:
# add staging data to metastore, so that it can be queries
add_to_metastore(table_name='bike_details_staging', path=stagingPath)

In [0]:
%sql

SELECT COUNT(1) FROM bike_details_staging

count(1)
25


In [0]:
from datetime import datetime
# Read staging dataframe to apply upsert operation
staging_df = spark.read.table("bike_details_staging").filter("p_incident_date == '{}'".format(datetime.today().strftime('%Y-%m-%d')))

In [0]:
# Performs UPSERTS (Update or Insert) and creates processed data
insert_update_processed(staging_df)

In [0]:
# add processed data to metastore, so that it can be queries
add_to_metastore(table_name='bike_details_processed', path=processedPath)

In [0]:
%sql

SELECT COUNT(1) FROM bike_details_processed

count(1)
25
